## Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

## Read all files

In [2]:
#read cv file (pdf or txt)
with open('cv/cv.txt','rb') as file:
    cv = file.read()

In [3]:
#read d.scientist csv
ds = pd.read_csv('jobs data/data_scientist.csv')
ds.head()

,Title,Company,Description
0,Data Scientist,OSTC,"Company Description\r\nIn just over 15 years, ..."
1,Biostatistician / Data Scientist - Innovative ...,Warman O'Brien,Biostatistician / Data Scientist - London\r\nI...
2,Data Scientist,ctrlio Ltd,ctrlio helps motor insurers win customers more...
3,Junior Data Scientist,Novafutur,This is an opportunity to become a Data Scient...
4,Data Scientist Lead,"JPMorgan Chase Bank, N.A.",Position Overview & Responsibilities:\r\nThis ...


In [4]:
#read m.learning engineers csv
ml = pd.read_csv('jobs data/ml.csv')
ml.head()

,Title,Company,Description
0,Machine Learning Engineer Graduate Scheme,Kubrick Group,Would you love to explore a career in Data Sci...
1,Machine Learning Engineer-Trust & Safety,TikTok,Responsibilities\r\nTikTok is the leading dest...
2,"Engineer, Machine Learning",OakNorth Bank,OakNorth is the next-generation credit and mon...
3,Machine Learning Research Engineer,Twitter,Who we are\r\nAre you interested in helping bu...
4,CIB Applied AI & Machine Learning - Economic a...,"JPMorgan Chase Bank, N.A.",About Data Analytics within Global Research\r\...


## Get desc of the jobs

In [5]:
#trim skills and reqs from desc of jobs
def empty_count(str_series):
    counter = 0
    for ind in str_series:
        if ind == []:
            counter+=1
    return counter
ds_desc=ds.Description.str.lower()
ml_desc=ml.Description.str.lower()
ds_qualifs=ds_desc.str.findall(r'((?:qualifications|requirements|desirable|skills|required|role)+(?:\:|\?|\n)(?:.|\s)*)')
ml_qualifs=ml_desc.str.findall(r'((?:qualifications|requirements|desirable|skills|required|role)+(?:\:|\?|\n)(?:.|\s)*)')
empty_count(ml_qualifs)/len(ml_qualifs)*100, empty_count(ds_qualifs)/len(ds_qualifs)*100

(69.44444444444444, 79.02439024390245)

In [6]:
#concat
ml_desc_ready=ml_desc.str.cat(sep=';;;')
ds_desc_ready=ds_desc.str.cat(sep=';;;')

## Calculate similarity between cv and jobs desc's

In [7]:
#Make a text corpus containing all words of documents. Use tokenisation and stop word removal.
import string
ds_lowers = ds_desc_ready.lower()
ml_lowers = ml_desc_ready.lower()
cv_lowers = cv.lower()
cv_lowers = str(cv_lowers)

#punctuation
translator = str.maketrans('', '', string.punctuation)
ds_clean = ds_lowers.translate(translator)
ml_clean = ml_lowers.translate(translator)
cv_clean = cv_lowers.translate(translator)

#tokenize
ds_tokens = nltk.word_tokenize(ds_clean)
ml_tokens = nltk.word_tokenize(ml_clean)
cv_tokens = nltk.word_tokenize(cv_clean)


# Use stopwords
from nltk.corpus import stopwords
ds_filtered = [w for w in ds_tokens if not w in stopwords.words('english')]
ml_filtered = [w for w in ml_tokens if not w in stopwords.words('english')]
cv_filtered = [w for w in cv_tokens if not w in stopwords.words('english')]
ds = ' '.join(ds_filtered)
ml = ' '.join(ml_filtered)
cv = ' '.join(cv_filtered)


#Stemming is optional
'''
from nltk.stem.porter import *
from collections import Counter
stemmer = PorterStemmer()
ds_stemmed = []
for item in ds_filtered:
    ds_stemmed.append(stemmer.stem(item))
ml_stemmed = []
for item in ml_filtered:
    ml_stemmed.append(stemmer.stem(item))
ds = ' '.join([el[0] for el in Counter(ds_stemmed).most_common(200)])
ml = ' '.join([el[0] for el in Counter(ml_stemmed).most_common(200)])
'''

"\nfrom nltk.stem.porter import *\nfrom collections import Counter\nstemmer = PorterStemmer()\nds_stemmed = []\nfor item in ds_filtered:\n    ds_stemmed.append(stemmer.stem(item))\nml_stemmed = []\nfor item in ml_filtered:\n    ml_stemmed.append(stemmer.stem(item))\nds = ' '.join([el[0] for el in Counter(ds_stemmed).most_common(200)])\nml = ' '.join([el[0] for el in Counter(ml_stemmed).most_common(200)])\n"

In [8]:
#Convert the documents into tf-idf vectors
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words='english')
docs = [ds,ml,cv]

In [9]:
#Find the cosine-similarity between them or any new document for similarity measure.
tfidf = vect.fit_transform(docs)
similarity = tfidf*tfidf.T
similarity=similarity.toarray()

#another method
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf)

In [10]:
docs_name = ['ds','ml','cv']
results = pd.DataFrame(data=similarity,index=docs_name,columns=docs_name)
results

,ds,ml,cv
ds,1.000000,0.859783,0.276385
ml,0.859783,1.000000,0.254288
cv,0.276385,0.254288,1.000000


As a result, I should improve my resume for these fields.